# 141B Final Project: Vaccine Hesitancy in the United States
## by Antonio Pelayo, Gianni Spiga, and Sharon Vien

In this notebook, we explore data provided by the CDC in regards to the hesistancy of adults in the United States in hopes to find any patterns or trends that might help better prepare for more efficient vaccine rollout on a nationwide and statewide scale in the future.

## Grading Criteria 
1. Project organization, writeup readability, and overall conclusions
2. Code quality, readability, and efficiency 
3. Data Munging
4. Data Visualizations
5. Data Extraction

## Table of Contents
* [Data Extraction](#data-extraction)
* [Variable Descriptions](#variable-description)
* [Data Munging and Cleaning](#data-munging)
* [Data Visualization](#data-visualization)
* [Conclusions](#conclusions)

In [8]:
# AutoFormatting for all code
%load_ext nb_black

<IPython.core.display.Javascript object>

In [9]:
# Libraries
import datetime
import requests
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sqlalchemy as sqla
import plotnine as p9
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from bioinfokit.analys import stat

# Other data used in the analysis
county_deaths_df = pd.read_csv("data/cdc-deaths-by-county-and-race.csv")
state_abbreviations_df = pd.read_csv("data/state-abbreviations.csv")
state_parties_df = pd.read_csv("data/state-political-parties.csv")
state_populations_df = pd.read_csv("data/census-2020-county-population-totals.csv")

<IPython.core.display.Javascript object>

<a id="data-extraction"></a>
### Extracting the Data

In [ ]:
# Request hesitancy data
hesitancy_endpoint = 'https://data.cdc.gov/resource/q9mh-h2tw.json?$limit=4000'

r = requests.get(hesitancy_endpoint)
hesitancy_df = pd.DataFrame(r.json())
hesitancy_df.head()

In [ ]:
# Request vaccination data
cdc_completed_vaccinations_endpoint = 'https://data.cdc.gov/resource/8xkx-amqh.json?$limit=5000'
r = requests.get(cdc_completed_vaccinations_endpoint)
county_vaccination_counts_df = pd.DataFrame(r.json())
# county_vaccination_counts_df.head()
county_vaccination_counts_df = county_vaccination_counts_df.filter(
    [
        'date',
        'recip_state',
        'series_complete_yes'
    ]
)

# Filter by today's date in format YYYY-MM-DD
today = datetime.datetime(2021, 12, 9).strftime('%Y-%m-%d')
today = today + 'T00:00:00.000'
county_vaccination_counts_df = county_vaccination_counts_df[
    county_vaccination_counts_df['date'] == today
]

# Turn completed vaccination counts into integers
county_vaccination_counts_df['series_complete_yes'] = county_vaccination_counts_df['series_complete_yes'].astype(int)
county_vaccination_counts_df['State Abbreviation'] = county_vaccination_counts_df['recip_state'].str.upper()

# Group counts by state
state_vaccinations_df = county_vaccination_counts_df.groupby('State Abbreviation').sum()
state_vaccinations_df.head()

<a id="variable-description"></a>
### Addressing and Understanding our Variables

In [ ]:
hesitancy_df.info()

#### Variable Definitions
1. fips_code = numbers that uniquely identify geographic areas  
2. county_name = name of county  
3. state = name of state  
4. estimated_hesitant = percent of population that indicated they would "probably not" or "definitely not" receive a COVID-19 vaccine when available  
5. estimated_hesitant_or_unsure = percent of population that indicated they would "probably not" or "unsure" or "definitely not" receive a COVID-19 vaccine when available  
6. estimated_strongly_hesitant = percent of population that indicated they would "definitely not" receive a COVID-19 vaccine when available  
7. social_vulnerability_index = the extent to which a community is socially vulnerable to disaster. The factors considered in developing the SVI include economic data as well as data regarding education, family characteristics, housing language ability, ethnicity, and vehicle access.   
8. svi_category = low, moderate, high, very high vulnerability   
9. ability_to_handle_a_covid = ability to handle Covid-19 outbreak in percent  
10. cvac_category = level of concern for vaccine rollout   
11. percent_adults_fully = percent of adults fully vaccinated  
12. percent_hispanic = percent of hispanic adults fully vaccinated   
13. percent_non_hispanic_american = percent of non hispanic american adults fully vaccinated    
14. percent_non_hispanic_asian = percent of non hispanic asian adults fully vaccinated  
15. percent_non_hispanic_black = percent of non hispanic black adults fully vaccinated  
16. percent_non_hispanic_native = percent of non hispanic native adults fully vaccinated  
17. percent_non_hispanic_white = percent of non hispanic white adults fully vaccinated  
18. geographical point = where the county lies  
19. state_code = code of the state  
20. county_boundary = coordinates of boundary  
21. state_boundary = coordinate of state boundary

<a id="data-munging"></a>
### Data Munging and Cleaning

#### Hesitancy of vaccines by political parties in the US

In [ ]:
# Filter hesitancy data
hesitancy_by_state_df = hesitancy_df.filter(
    [
        'state', 
        'estimated_hesitant',
        'estimated_hesitant_or_unsure',
        'estimated_strongly_hesitant'
    ]
)

# Set values to numeric
hesitancy_by_state_df['estimated_hesitant'] = hesitancy_by_state_df['estimated_hesitant'].astype(float)
hesitancy_by_state_df['estimated_hesitant_or_unsure'] = hesitancy_by_state_df['estimated_hesitant_or_unsure'].astype(float)
hesitancy_by_state_df['estimated_strongly_hesitant'] = hesitancy_by_state_df['estimated_strongly_hesitant'].astype(float)

# Calculate non-hesitant proportion
hesitancy_by_state_df['estimated_not_hesitant'] = 1 - (
    hesitancy_by_state_df['estimated_hesitant'] + hesitancy_by_state_df['estimated_hesitant_or_unsure'] + hesitancy_by_state_df['estimated_strongly_hesitant']
)

# Gropu by state
hesitancy_by_state_df = hesitancy_by_state_df.groupby('state').mean()

# Attatch state Governer's party affiliaition
state_parties_df['State'] = state_parties_df['Location'].apply(
    lambda x: x.upper() # Uppercase all state names
)
state_parties_df.drop(columns=['Location'], inplace=True) 
state_parties_df.set_index('State', inplace=True)

hesitancy_by_state_df = hesitancy_by_state_df.merge(state_parties_df, left_index=True, right_index=True)
hesitancy_by_state_df.head()


#### Proportions of COVID-19 deaths vs proportion of state population vaccinated

In [ ]:
# Filter for states with at least 100 completed vaccinations
state_vaccinations_df = state_vaccinations_df[
    state_vaccinations_df['series_complete_yes'] >= 100
]

# Join with population data for each state
state_populations_df = state_populations_df[
    state_populations_df['STNAME'] == state_populations_df['CTYNAME']
]

# Filter columns 
state_populations_df['State Name'] = state_populations_df['STNAME'].str.upper()
state_populations_df = state_populations_df.filter(
    ['State Name', 'POPESTIMATE2020']
)

# Sum county populations
state_populations_df = state_populations_df.groupby('State Name').sum()   

# Merge vaccination counts with population data on state abbreviations
state_populations_df = state_populations_df.join(
    state_abbreviations_df.set_index('State Name'), 
    on='State Name'
)

# Join
state_vaccinations_df = state_vaccinations_df.join(
    state_populations_df.reset_index().set_index('State Abbreviation'),
).dropna(how='any')

# # Calculate vaccinated proportion
state_vaccinations_df['Vaccine Proportion'] = state_vaccinations_df['series_complete_yes'] / state_vaccinations_df['POPESTIMATE2020']

# Calculate proportion of COVID-19 deaths
county_deaths_df = county_deaths_df.iloc[::3]

state_deaths_df = county_deaths_df.groupby('State').sum()
state_deaths_df = state_deaths_df.filter(['Total deaths', 'COVID-19 Deaths'])

state_deaths_df['COVID-19 Death Proportion'] = state_deaths_df['COVID-19 Deaths'] / state_deaths_df['Total deaths']
state_deaths_df.index.rename('State Abbreviation', inplace=True)

state_vaccinations_df = state_vaccinations_df.join(
    state_deaths_df
)

state_vaccinations_df = state_vaccinations_df.filter(
    [
        'State Name', 
        'series_complete_yes', 
        'POPESTIMATE2020', 
        'Vaccine Proportion', 
        'COVID-19 Deaths', 
        'Total deaths',
        'COVID-19 Death Proportion'
    ]
)
state_vaccinations_df.head()

#### Vaccine Hesitancy and Vaccination Rate 

In [ ]:
import warnings
warnings.filterwarnings("ignore")

#Create new dataframe for vaccine hesitancy in states
hesitancy_vaccination = hesitancy_df[
    [
        "fips_code",
        "county_name",
        "state",
        "ability_to_handle_a_covid",
        "social_vulnerability_index",
        "cvac_category",
        "estimated_hesitant",
        "estimated_hesitant_or_unsure",
        "estimated_strongly_hesitant",
        "percent_adults_fully",
    ]
]
#Change to numeric
hesitancy_vaccination["ability_to_handle_a_covid"] = hesitancy_vaccination["ability_to_handle_a_covid"].astype(float)
hesitancy_vaccination["social_vulnerability_index"] = hesitancy_vaccination["social_vulnerability_index"].astype(float)
hesitancy_vaccination["estimated_hesitant"] = hesitancy_vaccination["estimated_hesitant"].astype(float)
hesitancy_vaccination["estimated_hesitant_or_unsure"] = hesitancy_vaccination["estimated_hesitant_or_unsure"].astype(float)
hesitancy_vaccination["estimated_strongly_hesitant"] = hesitancy_vaccination["estimated_strongly_hesitant"].astype(float)
hesitancy_vaccination["percent_adults_fully"] = hesitancy_vaccination["percent_adults_fully"].astype(float)
hesitancy_vaccination_state_df = hesitancy_vaccination.groupby('state').mean()
hesitancy_vaccination_state_df = hesitancy_vaccination_state_df.reset_index()
hesitancy_vaccination_state_df['not_hesitant'] = 1 - (hesitancy_vaccination_state_df["estimated_strongly_hesitant"] + hesitancy_vaccination_state_df["estimated_hesitant_or_unsure"] + hesitancy_vaccination_state_df["estimated_hesitant"])
hesitancy_vaccination_state_df['total_hesitancy'] = hesitancy_vaccination_state_df["estimated_strongly_hesitant"] + hesitancy_vaccination_state_df["estimated_hesitant_or_unsure"] + hesitancy_vaccination_state_df["estimated_hesitant"]


<a id="data-visualization"></a>
### Visualizing the Data

We will explore numerous visualizations such as scatterplots, boxplots, and density graphs to better understand the data. 

#### Boxplot of hesitancy estimates by political party

In [ ]:
# Boxplot of hesitancy estimates by political party 
hesitancy_by_state_df.boxplot(
    column=[
        'estimated_hesitant', 
        'estimated_hesitant_or_unsure', 
        'estimated_strongly_hesitant',
        'estimated_not_hesitant'
    ], 
    by='Governor Political Affiliation',
    figsize=(10, 6)
)

These boxplots show that Republicans are much more likely to be hesitant towards the COVID-19 vaccine than Democrats.
In other notebooks we look into death proportions by state and political party to see if there is a difference in proportion of deaths due to COVID-19. It would make sense to see a higher proportion of COVID-19 deaths in Republican governed states due to the hesitancy and lack of vaccinations.

#### Boxplot of COVID-19 Death proportions vs Vaccine proportions by state

In [ ]:
# Plot boxplot
state_vaccinations_df.plot(
    x='Vaccine Proportion',
    y='COVID-19 Death Proportion',
    kind='scatter',
    title='COVID-19 Death Proportions by Vaccine Proportion',
    figsize=(10, 6)
)

# Annotate points
for idx, row in state_vaccinations_df.iterrows(): 
    plt.text(row['Vaccine Proportion'], row['COVID-19 Death Proportion'], idx)

#### Scatterplot of Vaccine Hesitancy and Vaccine Rate by States

In [ ]:
import plotnine as p9
    
gg = p9.ggplot(hesitancy_vaccination_state_df)
gg += p9.aes(
    x="percent_adults_fully", y="total_hesitancy", color="state"
)  # add color mapping by State
gg += p9.geom_point()
gg

In [ ]:
# Compute correlation coefficient
death_vax_correlation = state_vaccinations_df['Vaccine Proportion'].corr(state_vaccinations_df['COVID-19 Death Proportion'])
print(f'Correlation coefficient: {death_vax_correlation}')

This scatter plot and negative correlation coeffifient indicates that the proportion of COVID-19 deaths decreases as the proportion of people vaccinated increases. This is a good indication on the benefits of vaccination.

Hawaii was the only state ommited from the analysis due to the lack of vaccination data. 

In [ ]:
# Compute correlation coefficient between Vaccine Hesitancy and Vaccination Rate 
hesitancy_vax_correlation = hesitancy_vaccination_state_df['percent_adults_fully'].corr(hesitancy_vaccination_state_df['total_hesitancy'])
print(f'Correlation coefficient: {hesitancy_vax_correlation}')

This scatter plot and negative correlation coeffifient indicates as vaccine hesitancy increases, the percentage of adults fully vaccinated decreases. However, in the hesitancy and actually vaccination rate notebook, we will dive deeper into this question to find whether other variables influence vaccine hesitancy and vaccination rate.

### Data Analysis

#### Vaccine Hesitancy based on Governor's Political Affliation

In [ ]:
# Extracting political party data from our public github
url = "https://raw.githubusercontent.com/gspiga/STAT141Bfinal/main/data/raw_data.csv"
poli_df = pd.read_csv(url)
poli_df.head()

<a id='conclusions'></a>
### Conclusions 